# 📓 Notebook - Teori & Praktik Bab 12: Model Kustom & Pelatihan dengan TensorFlow

Bab ini berfokus pada fleksibilitas TensorFlow. Meskipun API Keras standar sudah mencakup sebagian besar kebutuhan, terkadang diperlukan kendali lebih dalam untuk kasus khusus, seperti loss function yang unik, struktur layer yang tidak biasa, atau loop pelatihan manual.

Topik pembahasan meliputi:
1.  **Tensor & Operasi**: Struktur data dasar TensorFlow.
2.  **Custom Loss Function**: Membuat fungsi kerugian sendiri.
3.  **Custom Metrics**: Membuat metrik evaluasi sendiri (Stateful Metrics).
4.  **Custom Layers**: Membuat layer jaringan saraf sendiri.
5.  **Custom Models**: Membuat arsitektur model sendiri.
6.  **Custom Training Loop**: Melatih model secara manual menggunakan `GradientTape`.
7.  **TensorFlow Functions**: Optimasi performa graf dengan `@tf.function`.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.20.0


## 1. Tensor dan Operasi Dasar

### Teori
Tensor adalah struktur data utama dalam TensorFlow, mirip dengan array NumPy, tetapi memiliki kemampuan untuk diproses oleh GPU/TPU dan merupakan bagian dari graf komputasi. Tensor bersifat *immutable* (tidak dapat diubah setelah dibuat), kecuali `tf.Variable` yang digunakan untuk menyimpan bobot model.

Operasi matematika pada tensor dilakukan menggunakan fungsi-fungsi `tf.*` (misalnya `tf.add`, `tf.square`, `tf.matmul`).

In [3]:
# Pembuatan Tensor Konstan
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

# Operasi Dasar
print("Tensor t:\n", t)
print("Ukuran (Shape):", t.shape)
print("Tipe Data:", t.dtype)

# Operasi Matematika
print("\nPenjumlahan (t + 10):\n", t + 10)
print("Kuadrat (tf.square(t)):\n", tf.square(t))
print("Perkalian Matriks (t @ t.T):\n", t @ tf.transpose(t))

# Konversi NumPy ke Tensor dan sebaliknya
a = np.array([2., 4., 5.])
tf_a = tf.constant(a)
print("\nNumPy ke Tensor:", tf_a)
print("Tensor ke NumPy:", tf_a.numpy())

Tensor t:
 tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
Ukuran (Shape): (2, 3)
Tipe Data: <dtype: 'float32'>

Penjumlahan (t + 10):
 tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
Kuadrat (tf.square(t)):
 tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
Perkalian Matriks (t @ t.T):
 tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)

NumPy ke Tensor: tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
Tensor ke NumPy: [2. 4. 5.]


## 2. Custom Loss Function

### Teori
Ketika fungsi *loss* bawaan (seperti MSE atau CrossEntropy) tidak sesuai dengan masalah yang dihadapi, fungsi kustom dapat dibuat. 

Fungsi *loss* kustom harus menerima dua argumen: `y_true` (label asli) dan `y_pred` (prediksi model), serta mengembalikan nilai error per instans data. Contoh di bawah mengimplementasikan **Huber Loss**, yang lebih tahan terhadap *outlier* dibandingkan Mean Squared Error.

In [6]:
def custom_huber_loss(y_true, y_pred):
    threshold = 1.0
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    
    # Jika error kecil: kuadratkan (seperti MSE)
    squared_loss = tf.square(error) / 2
    # Jika error besar: linear (seperti MAE)
    linear_loss = threshold * tf.abs(error) - threshold**2 / 2
    
    return tf.where(is_small_error, squared_loss, linear_loss)

# Pengujian Fungsi Loss
y_true = tf.constant([1.0])
y_pred = tf.constant([2.5]) # Error = 1.5 (Besar)
print("Huber Loss (Error Besar):", custom_huber_loss(y_true, y_pred).numpy())

# Penggunaan dalam Model Keras
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[1])])
model.compile(loss=custom_huber_loss, optimizer="sgd")
# model.fit(...) # Dapat digunakan langsung saat training

Huber Loss (Error Besar): [1.]


## 3. Custom Metrics (Stateful)

### Teori
Berbeda dengan *Loss* yang dihitung per *batch*, Metrik (seperti Akurasi atau Precision) seringkali perlu diakumulasikan di seluruh dataset (Stateful). 

Untuk membuat metrik kustom yang *stateful*, kelas `keras.metrics.Metric` harus di-*subclass*. Metode `update_state` digunakan untuk memperbarui akumulasi, dan `result` untuk menghitung hasil akhir.

In [8]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = custom_huber_loss
        
        # PERBAIKAN: Gunakan keyword argument 'name', 'shape', dan 'initializer' secara eksplisit
        self.total = self.add_weight(name="total", shape=(), initializer="zeros")
        self.count = self.add_weight(name="count", shape=(), initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count

    def reset_states(self):
        self.total.assign(0.)
        self.count.assign(0.)

# Pengujian Metrik
m = HuberMetric()
m.update_state(tf.constant([1.0]), tf.constant([2.5]))
print("Hasil Akumulasi Metrik:", m.result().numpy())

Hasil Akumulasi Metrik: 1.0


## 4. Custom Layers

### Teori
Jika layer standar (Dense, Conv2D, Dropout) tidak cukup, layer kustom dapat dibuat dengan menurunkan kelas `keras.layers.Layer`.

Metode penting:
* `__init__`: Menerima hyperparameter.
* `build(input_shape)`: Membuat bobot (weights) saat bentuk input diketahui.
* `call(inputs)`: Melakukan operasi perhitungan (forward pass).

In [9]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        # Membuat kernel (weights) dan bias
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        # Operasi matriks: X * W + b
        return self.activation(X @ self.kernel + self.bias)

# Menggunakan Layer Kustom
model_custom = keras.models.Sequential([
    keras.layers.Input(shape=(5,)), # Input shape explicit
    MyDense(10, activation="relu"),
    MyDense(1)
])

print("Output dari model dengan Custom Layer:")
print(model_custom(tf.random.uniform((2, 5))))


Output dari model dengan Custom Layer:
tf.Tensor(
[[0.1977665]
 [0.5968505]], shape=(2, 1), dtype=float32)


## 5. Custom Models

### Teori
Untuk arsitektur yang kompleks (misalnya yang memiliki skip connection atau input ganda yang dinamis), kelas `keras.Model` dapat di-*subclass*.

Layer didefinisikan di `__init__`, dan alur data didefinisikan di `call`. Ini memberikan fleksibilitas penuh seperti pemrograman Python biasa (bisa menggunakan `if`, `for`, dll dalam `call`).

In [10]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        # Membuat list hidden layers
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # Skip connection (menambahkan input asli ke output)

# Menggunakan Custom Block dalam Model
model_res = keras.models.Sequential([
    keras.layers.Input(shape=(10,)),
    ResidualBlock(n_layers=2, n_neurons=10),
    keras.layers.Dense(1)
])

model_res.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ residual_block (ResidualBlock)  │ (None, 10)             │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 231 (924.00 B)

 Trainable params: 231 (924.00 B)

 Non-trainable params: 0 (0.00 B)

## 6. Custom Training Loops (GradientTape)

### Teori
Metode `.fit()` sangat kuat, tetapi terkadang peneliti membutuhkan kendali penuh atas setiap langkah pelatihan (misalnya untuk GANs atau Reinforcement Learning).

Ini dilakukan dengan **GradientTape**:
1.  Buka blok `with tf.GradientTape() as tape:`.
2.  Lakukan *forward pass* (prediksi).
3.  Hitung *loss*.
4.  Minta tape untuk menghitung gradien loss terhadap variabel model.
5.  Gunakan optimizer untuk mengupdate variabel berdasarkan gradien tersebut.

In [11]:
# Data Dummy Linear Regression
X_train = tf.random.normal(shape=(100, 1))
y_train = 3 * X_train + 2 + tf.random.normal(shape=(100, 1)) * 0.5

# Model Sederhana
model_manual = keras.layers.Dense(1, input_shape=[1])
optimizer = keras.optimizers.SGD(learning_rate=0.05)
loss_fn = keras.losses.MeanSquaredError()

print("Training Loop Manual dimulai...")
for epoch in range(5):
    with tf.GradientTape() as tape:
        # 1. Forward pass
        y_pred = model_manual(X_train)
        # 2. Hitung Loss
        loss = loss_fn(y_train, y_pred)
    
    # 3. Hitung Gradien
    gradients = tape.gradient(loss, model_manual.trainable_variables)
    
    # 4. Update Weights
    optimizer.apply_gradients(zip(gradients, model_manual.trainable_variables))
    
    print(f"Epoch {epoch+1}, Loss: {loss.numpy():.4f}")

# Hasil weights harus mendekati 3 dan bias mendekati 2
print("Weights & Bias hasil training:", model_manual.get_weights())

Training Loop Manual dimulai...
Epoch 1, Loss: 17.1847
Epoch 2, Loss: 14.3423
Epoch 3, Loss: 11.9761
Epoch 4, Loss: 10.0063
Epoch 5, Loss: 8.3664
Weights & Bias hasil training: [array([[0.46739513]], dtype=float32), array([0.7787275], dtype=float32)]


## 7. TensorFlow Functions (@tf.function)

### Teori
Python bersifat fleksibel tapi lambat. TensorFlow dapat mengubah fungsi Python menjadi **Graf TensorFlow** yang sangat cepat dan optimal untuk dijalankan di GPU/TPU menggunakan dekorator `@tf.function`.

Saat fungsi dihiasi dengan `@tf.function`, TensorFlow akan melacak eksekusi fungsi tersebut (Tracing) dan membuat versi graf statisnya.

In [12]:
@tf.function
def fungsi_cepat(x):
    print("Tracing fungsi... (hanya muncul saat kompilasi awal)")
    return tf.square(x) + 1

# Panggilan pertama: Tracing terjadi
print("Output 1:", fungsi_cepat(tf.constant(2.)))

# Panggilan kedua: Menggunakan graf yang sudah dicache (tidak ada print Tracing)
print("Output 2:", fungsi_cepat(tf.constant(5.)))

# Keras secara default menggunakan Graph Mode di belakang layar pada .fit()

Tracing fungsi... (hanya muncul saat kompilasi awal)
Output 1: tf.Tensor(5.0, shape=(), dtype=float32)
Output 2: tf.Tensor(26.0, shape=(), dtype=float32)


### Kesimpulan Bab 12

Bab ini memperlihatkan kekuatan TensorFlow di balik layar Keras:
* **Tensor** adalah fondasi data.
* **Custom Loss/Metric/Layer** memungkinkan adaptasi model untuk masalah spesifik.
* **GradientTape** memberikan kendali penuh atas alur matematika pelatihan.
* **@tf.function** digunakan untuk meningkatkan performa produksi.